In [35]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import joblib
from transformers import BertTokenizer
import torch
from transformers import Trainer, TrainingArguments
from sentence_transformers import SentenceTransformer
import numpy as np

In [39]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

# Load source files
movies_df = pd.read_csv("movies_with_plot.csv")
roles_df = pd.read_csv("malayalam_movie_cast_dataset.csv")

# Normalize movie names
movies_df["movie_name"] = movies_df["movie_name"].str.strip().str.lower()
roles_df["movie_name"] = roles_df["movie_name"].str.strip().str.lower()

# Merge dynamically
df = pd.merge(
    roles_df,
    movies_df,
    on=["movie_name", "year"],
    how="inner"
)



print(df.head())

           movie_name  year          actor_name  character_name  \
0  sufi paranja katha  2010  Sharbani Mukherjee  Karthi, Suhara   
1  sufi paranja katha  2010       Thampi Antony     Sanku Menon   
2  sufi paranja katha  2010        Prakash Bare        Mamootty   
3  sufi paranja katha  2010   Jagathy Sreekumar  Avaru Musaliar   
4  sufi paranja katha  2010     V. K. Sreeraman    Saidu Mullah   

                                                plot  
0  This film is a narrative by Sufi, a Muslim sch...  
1  This film is a narrative by Sufi, a Muslim sch...  
2  This film is a narrative by Sufi, a Muslim sch...  
3  This film is a narrative by Sufi, a Muslim sch...  
4  This film is a narrative by Sufi, a Muslim sch...  


In [44]:
# 1. Build input text
df["input_text"] = df["plot"] + " Character: " + df["character_name"]


# 2. Embed
model = SentenceTransformer("all-MiniLM-L6-v2")
train_embeddings = model.encode(df["input_text"].tolist(), show_progress_bar=True)

# 3. Save
np.save("final_embeddings.npy", train_embeddings)
df.to_csv("train_data.csv", index=False)



Loading weights: 100%|██████████| 103/103 [00:00<00:00, 742.03it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 291/291 [03:37<00:00,  1.34it/s]


In [45]:
def suggest_actors(user_plot, user_character, top_k=5):
    query_text = user_plot + " Character: " + user_character
    query_emb = model.encode([query_text])

    sims = cosine_similarity(query_emb, train_embeddings)[0]
    top_idx = np.argsort(sims)[-top_k:][::-1]

    return train_df.iloc[top_idx][["actor_name", "movie_name", "character_name"]]